# 앞에 부분에서 필요한 코드 불러오기

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.datasets import TranslationDataset, Multi30k
from torchtext import data 
from torchtext.data import Field, BucketIterator
# from torchtext.legacy.data import Field, BucketIterator

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time

/home/dsail/hyolim/hyolim_py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [3]:
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

SRC = data.Field(tokenize = tokenize_de, init_token = '<sos>', eos_token = '<eos>', lower = True, batch_first = True)
TRG = data.Field(tokenize = tokenize_en, init_token = '<sos>', eos_token = '<eos>', lower = True, batch_first = True)

train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), fields = (SRC, TRG))

SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [4]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128 # 배치 크기(batch size): 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
     batch_size = BATCH_SIZE,
     device = device)

In [5]:
for i, batch in enumerate(train_iterator):
    src = batch.src
    trg = batch.trg
    break
    
# src 정의하기
batch = next(iter(train_iterator))
print(type(batch))
print(batch)
src = batch.src
src
x=src
print(x.shape)
x

<class 'torchtext.data.batch.Batch'>

[torchtext.data.batch.Batch of size 128 from MULTI30K]
	[.src]:[torch.cuda.LongTensor of size 128x33 (GPU 1)]
	[.trg]:[torch.cuda.LongTensor of size 128x35 (GPU 1)]
torch.Size([128, 33])


tensor([[ 2,  8, 67,  ...,  1,  1,  1],
        [ 2, 43, 41,  ...,  1,  1,  1],
        [ 2,  8, 16,  ...,  1,  1,  1],
        ...,
        [ 2,  8, 36,  ...,  1,  1,  1],
        [ 2,  5, 13,  ...,  1,  1,  1],
        [ 2,  8, 16,  ...,  1,  1,  1]], device='cuda:1')

In [6]:
# 정의
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

input_dim=INPUT_DIM
hid_dim=HID_DIM
n_layers=ENC_LAYERS
n_heads=ENC_HEADS
pf_dim=ENC_PF_DIM
dropout=ENC_DROPOUT
device=device
max_length = 100
output_dim = OUTPUT_DIM

In [7]:
class EncoderLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, pf_dim,dropout, device):
        super().__init__()
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device) # 멀티-헤드 어텐션
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, pf_dim, dropout) # 위치인코더 -> 위치 인코딩       
        self.ff_layer_norm = nn.LayerNorm(hid_dim) # 덧셈과 계층 정규화 
        self.dropout = nn.Dropout(dropout)
          
    def forward(self, src, src_mask):
        _src, _ = self.self_attention(src, src, src, src_mask) ##########
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        _src = self.positionwise_feedforward(src)
        src = self.ff_layer_norm(src + self.dropout(_src))
        return src

In [8]:
class Encoder(nn.Module):
    def __init__(self,input_dim, hid_dim, n_layers, n_heads, pf_dim,dropout, device,max_length = 100):
        super().__init__()
        self.device = device        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, n_heads, pf_dim,dropout, device) for _ in range(n_layers)])
        self.dropout = nn.Dropout(dropout)
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        batch_size = src.shape[0]
        src_len = src.shape[1]
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        for layer in self.layers:
            src = layer(src, src_mask)
        return src # 마지막 레이어의 출력을 반환

In [9]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()        
        assert hid_dim % n_heads == 0    # assert: 가정 설정문, 뒤의 조건이 True가 아니면 error를 발생시킴
        self.hid_dim = hid_dim 
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads # d_model의 차원을 n_heads개로 나누어서 병렬 어텐션 수행.
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)        
        self.fc_o = nn.Linear(hid_dim, hid_dim) # WO(weight matrix)에 해당하는 밀집층 정의        
        self.dropout = nn.Dropout(dropout)        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)    
        
    def forward(self, query, key, value, mask = None): #######        
        batch_size = query.shape[0]                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale # Attention Energy 계산    
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        attention = torch.softmax(energy, dim = -1)
        x = torch.matmul(self.dropout(attention), V)
        x = x.permute(0, 2, 1, 3).contiguous()
        x = x.view(batch_size, -1, self.hid_dim)
        x = self.fc_o(x)
        return x, attention

In [10]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim) # fc를 두 번 거쳐서 입력과 출력 차원이 동일하도록(인풋의 shape를 그대로 유지함) 해줌        
        self.dropout = nn.Dropout(dropout)        
    def forward(self, x):
        x = self.dropout(torch.relu(self.fc_1(x)))
        x = self.fc_2(x)
        return x

In [11]:
class DecoderLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, pf_dim, dropout, device):
        super().__init__()
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)       
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, pf_dim, dropout)        
        self.ff_layer_norm = nn.LayerNorm(hid_dim)        
        self.dropout = nn.Dropout(dropout)

    def forward(self, trg, enc_src, trg_mask, src_mask):
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
        _trg = self.positionwise_feedforward(trg)
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        return trg, attention

## 2.6 디코더 아키텍처
* 하이퍼 파라미터(hyperparameter)
    * output_dim: 하나의 단어에 대한 원 핫 인코딩 차원
    * hidden_dim: 하나의 단어에 대한 임베딩 차원
    * n_layers: 내부적으로 사용할 인코더 레이어의 개수
    * n_heads: 헤드(head)의 개수 = scaled dot-product attention의 개수
    * pf_dim: Feedforward 레이어에서의 내부 임베딩 차원
    * dropout_ratio: 드롭아웃(dropout) 비율
    * max_length: 문장 내 최대 단어 개수
* 원본 논문과는 다르게 위치 임베딩(positional embedding)을 학습하는 형태로 구현.
    * 이는 BERT와 같은 모던 트랜스포머 아키텍처에서 사용되는 방식임.
* Seq2Seq과는 마찬가지로 실제로 추론(inference) 시기에서는 디코더를 반복적으로 넣을 필요가 있음.
    * 학습(training) 시기에서는 한 번에 출력 문장을 구해 학습 가능.
* 소스 문장의 pad 토큰에 대하여 마스크(mask) 값을 0으로 설정합니다.
* 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크를 사용.

In [12]:
class Decoder(nn.Module):
    def __init__(self, output_dim, hid_dim, n_layers, n_heads, pf_dim, dropout, device,max_length = 100):
        super().__init__()
        
        self.device = device
        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, n_heads, pf_dim, dropout, device) for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
                            
        #pos = [batch size, trg len]
            
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
                
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            # 소스 마스크와 타겟 마스크 모두 사용
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        
        #output = [batch size, trg len, output dim]
            
        return output, attention

### enc_src는 encoder를 거친 src이다

In [13]:
#  인코더 필요한 부분 정의
device = device
tok_embedding = nn.Embedding(input_dim, hid_dim).to(device) # Embedding(7853, 256)
pos_embedding = nn.Embedding(max_length, hid_dim).to(device) # Embedding(100, 256)
layers = nn.ModuleList([EncoderLayer(hid_dim, n_heads, pf_dim,0.1, device) for _ in range(n_layers)]).to(device) 
dropout = nn.Dropout(0.1).to(device) # Dropout(p=0.1, inplace=False)
scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device) # tensor([16.])

batch_size = src.shape[0] # 128
src_len = src.shape[1] # 

pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(device)
print(pos.shape)          #pos = [batch size, src len]

SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
src_pad_idx=SRC_PAD_IDX
src_mask = (src != src_pad_idx).unsqueeze(1).unsqueeze(2)
src_mask
src_mask.shape  #src_mask = [batch size, 1, 1, src len]

src = dropout((tok_embedding(src) * scale) + pos_embedding(pos))
src.shape

for layer in layers:
    src = layer(src, src_mask)
src 
src.shape

torch.Size([128, 33])


torch.Size([128, 33, 256])

### forward 시행 위한 enc_src 값 정의

In [14]:
encoder = Encoder
encoder = encoder

enc_src =src
print(enc_src.shape)

torch.Size([128, 33, 256])


### 디코더 아키텍처 이제 코드 하기
def __init__(self, output_dim, hid_dim, n_layers, n_heads, pf_dim, dropout, device,max_length = 100):

In [15]:
# 정의
device = device

tok_embedding = nn.Embedding(output_dim, hid_dim).to(device) # Embedding(5893, 256)
pos_embedding = nn.Embedding(max_length, hid_dim).to(device) # Embedding(100, 256)

layers = nn.ModuleList([DecoderLayer(hid_dim, n_heads, pf_dim, 0.1, device) for _ in range(n_layers)]).to(device)
fc_out = nn.Linear(hid_dim, output_dim).to(device) # Linear(in_features=256, out_features=5893, bias=True)
dropout = nn.Dropout(0.1).to(device)
scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)

trg_mask 정의 //  def make_trg_mask(self, trg):

In [16]:
# trg_mask 정의 //  def make_trg_mask(self, trg):
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
trg_pad_idx=TRG_PAD_IDX
print(trg_pad_idx)

1


In [17]:
print(trg.shape)
trg

torch.Size([128, 21])


tensor([[   2,   14,   13,  ...,    1,    1,    1],
        [   2,   19, 1693,  ...,    1,    1,    1],
        [   2,    4,  192,  ...,    1,    1,    1],
        ...,
        [   2,    4,   38,  ...,    1,    1,    1],
        [   2,    4,    9,  ...,    1,    1,    1],
        [   2,    9,   22,  ...,    1,    1,    1]], device='cuda:1')

In [18]:
trg_pad_mask = (trg != trg_pad_idx).unsqueeze(1).unsqueeze(2)
print((trg != trg_pad_idx).unsqueeze(1).shape)
print(trg_pad_mask.shape) #trg_pad_mask = [batch size, 1, 1, trg len]
trg_pad_mask 

torch.Size([128, 1, 21])
torch.Size([128, 1, 1, 21])


tensor([[[[ True,  True,  True,  ..., False, False, False]]],


        [[[ True,  True,  True,  ..., False, False, False]]],


        [[[ True,  True,  True,  ..., False, False, False]]],


        ...,


        [[[ True,  True,  True,  ..., False, False, False]]],


        [[[ True,  True,  True,  ..., False, False, False]]],


        [[[ True,  True,  True,  ..., False, False, False]]]], device='cuda:1')

* torch.trill
    * 행렬의 아래쪽 삼각형 부분을 인풋으로 반환하고 다른 요소는 0으로 바꿈
* torch.ones
    * 1로 찬 텐서 반환

In [19]:
#torch.ones((trg_len, trg_len)).shape

In [20]:
trg_len = trg.shape[1]
trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = device)).bool()
print(trg_sub_mask.shape) #trg_sub_mask = [trg len, trg len]
trg_sub_mask

torch.Size([21, 21])


tensor([[ True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False],
        [ True,  True, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False],
        [ True,  True,  True, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False],
        [ True,  True,  True,  True, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False],
        [ True,  True,  True,  True,  True, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False],
        [ True,  True,  True,  True,  True,  True, False, False, False, False,
         False, False, False, False, False, Fa

In [21]:
trg_mask = trg_pad_mask & trg_sub_mask # True, False로 이루어진 텐서
print(trg_mask.shape) #trg_mask = [batch size, 1, trg len, trg len]

torch.Size([128, 1, 21, 21])


def forward(self, trg, enc_src, trg_mask, src_mask):

In [22]:
# forward 시행 전 shape 확인하기

print(trg.shape)            #trg = [batch size, trg len],trg = batch.trg 라고 위에서 정의함
print(enc_src.shape)        #enc_src = [batch size, src len, hid dim]
print(trg_mask.shape)        #trg_mask = [batch size, 1, trg len, trg len]
print(src_mask.shape)        #src_mask = [batch size, 1, 1, src len]

torch.Size([128, 21])
torch.Size([128, 33, 256])
torch.Size([128, 1, 21, 21])
torch.Size([128, 1, 1, 33])


In [23]:
# 1.
batch_size = trg.shape[0]
trg_len = trg.shape[1]

In [24]:
# 2.
pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(device)
print(pos.shape) #pos = [batch size, trg len]

torch.Size([128, 21])


In [25]:
# 3.
trg = dropout((tok_embedding(trg) * scale) + pos_embedding(pos))
print(trg.shape) #trg = [batch size, trg len, hid dim]

torch.Size([128, 21, 256])


In [26]:
# 4.
for layer in layers:
    # 소스 마스크와 타겟 마스크 모두 사용
    trg, attention = layer(trg, enc_src, trg_mask, src_mask)
    
print(trg.shape) #trg = [batch size, trg len, hid dim]
print(attention.shape) #attention = [batch size, n heads, trg len, src len]

torch.Size([128, 21, 256])
torch.Size([128, 8, 21, 33])


In [27]:
layers 

ModuleList(
  (0): DecoderLayer(
    (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (self_attention): MultiHeadAttentionLayer(
      (fc_q): Linear(in_features=256, out_features=256, bias=True)
      (fc_k): Linear(in_features=256, out_features=256, bias=True)
      (fc_v): Linear(in_features=256, out_features=256, bias=True)
      (fc_o): Linear(in_features=256, out_features=256, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder_attention): MultiHeadAttentionLayer(
      (fc_q): Linear(in_features=256, out_features=256, bias=True)
      (fc_k): Linear(in_features=256, out_features=256, bias=True)
      (fc_v): Linear(in_features=256, out_features=256, bias=True)
      (fc_o): Linear(in_features=256, out_features=256, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (enc_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (positionwise_feedforward): PositionwiseFeedforwardLayer(
   

In [28]:
# 5.
output = fc_out(trg)

# return 값
print(output.shape) #output = [batch size, trg len, output dim]
print(attention.shape)

torch.Size([128, 21, 5893])
torch.Size([128, 8, 21, 33])


## 2.7 디코더(Decoder) 레이어
* 하나의 디코더 레이어에 대해
    * 입력과 출력 차원이 같다.
    * 디코더 레이어를 여러 번 중첩해 사용.
    * 두 개의 Multi-Head Attention 레이어 사용.
* 하이퍼 파라미터(hyperparameter)
    * hidden_dim: 하나의 단어에 대한 임베딩 차원
    * n_heads: 헤드(head)의 개수 = scaled dot-product attention의 개수
    * pf_dim: Feedforward 레이어에서의 내부 임베딩 차원
    * dropout_ratio: 드롭아웃(dropout) 비율
* 소스 문장의 pad 토큰에 대하여 마스크(mask) 값을 0으로 설정합니다.
* 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크를 사용.

In [29]:
class DecoderLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, pf_dim, dropout, device):
        super().__init__()
        
        ''' Multi Head self Attention'''
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)

        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        
        ''' Encoder-decoder attention'''
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        
        ''' Positionwise FeedForward Layer'''        
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, pf_dim, dropout)
        
        self.ff_layer_norm = nn.LayerNorm(hid_dim)        
        
        self.dropout = nn.Dropout(dropout)
        
    # 인코더의 출력 값(enc_src)을 어텐션(attention)하는 구조    
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
        
        #self attention: 자기 자신에 대하여 어텐션(attention)
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
            
        #encoder attention: 디코더의 쿼리(Query)를 이용해 인코더를 어텐션(attention)
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        
        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
                    
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention

def __init__(self, hid_dim, n_heads, pf_dim, dropout, device):

In [30]:
# 정의
self_attn_layer_norm = nn.LayerNorm(hid_dim).to(device) # LayerNorm((256,), eps=1e-05, elementwise_affine=True)
self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, 0.1, device).to(device)
encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, 0.1, device).to(device)
enc_attn_layer_norm = nn.LayerNorm(hid_dim).to(device) # LayerNorm((256,), eps=1e-05, elementwise_affine=True)
positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, pf_dim, 0.1).to(device)
ff_layer_norm = nn.LayerNorm(hid_dim).to(device) # LayerNorm((256,), eps=1e-05, elementwise_affine=True)        
dropout = nn.Dropout(0.1).to(device)

* 인코더의 출력 값(enc_src)을 어텐션(attention)하는 구조    
    def forward(self, trg, enc_src, trg_mask, src_mask):

In [31]:
# forward 시행 전 shape 맞는지 확인
print(trg.shape)        #trg = [batch size, trg len, hid dim]
print(enc_src.shape)        #enc_src = [batch size, src len, hid dim]
print(trg_mask.shape)        #trg_mask = [batch size, 1, trg len, trg len]
print(src_mask.shape)        #src_mask = [batch size, 1, 1, src len]

torch.Size([128, 21, 256])
torch.Size([128, 33, 256])
torch.Size([128, 1, 21, 21])
torch.Size([128, 1, 1, 33])


* 디코더의 마스크드 셀프 어텐션
    * Query =Key= Value
    - Padding mask 마스킹은 어떻게? 어텐션 스코어 행렬의 마스킹 위치에 매우 작은 음수값을 넣어주어서
    - 추후, 어텐션 스코어 함수가 소프트맥스를 지난 후에 작은 음수 값으로 인해 0에 가까운 값이 되고 단어 간 유사도 구하는 일에 쓰이지 않게 된다

    - 앞쪽에 등장했던 단어들만 참고할 수 있도록, 뒤쪽에 나오는 단어가 무엇인지 참고하면 베끼는 것이 되어서! → 정상적인 모델의 학습을 유도)

In [32]:
self_attention

MultiHeadAttentionLayer(
  (fc_q): Linear(in_features=256, out_features=256, bias=True)
  (fc_k): Linear(in_features=256, out_features=256, bias=True)
  (fc_v): Linear(in_features=256, out_features=256, bias=True)
  (fc_o): Linear(in_features=256, out_features=256, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [33]:
# 1. 
# self attention: 자기 자신에 대하여 어텐션(attention)
_trg, _ = self_attention(trg, trg, trg, trg_mask)
print(_trg.shape)

torch.Size([128, 21, 256])


In [34]:
self_attn_layer_norm

LayerNorm((256,), eps=1e-05, elementwise_affine=True)

In [35]:
# 2.
# 드롭아웃, 잔차연결, 정규화
trg = self_attn_layer_norm(trg + dropout(_trg))
print(trg.shape) #trg = [batch size, trg len, hid dim]

torch.Size([128, 21, 256])


* 디코더의 인코더-디코더 어텐션
    * Query : 디코더 벡터 / Key= Value : 인코더 벡터
    * encoder attention: 디코더의 쿼리(Query)를 이용해 인코더를 어텐션(attention)
    *  디코더의 인코더-디코더 어텐션 : Query : 디코더 벡터 / Key= Value : 인코더 벡터
    *  -> 그래서 trg는 디코더 벡터에서 온 쿼리, enc_src, enc_src는 인코더에서 온 키와 벡터

In [36]:
encoder_attention

MultiHeadAttentionLayer(
  (fc_q): Linear(in_features=256, out_features=256, bias=True)
  (fc_k): Linear(in_features=256, out_features=256, bias=True)
  (fc_v): Linear(in_features=256, out_features=256, bias=True)
  (fc_o): Linear(in_features=256, out_features=256, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [37]:
# 3.
_trg, attention = encoder_attention(trg, enc_src, enc_src, src_mask)
print(_trg.shape)
print(attention.shape)

torch.Size([128, 21, 256])
torch.Size([128, 8, 21, 33])


In [38]:
enc_attn_layer_norm

LayerNorm((256,), eps=1e-05, elementwise_affine=True)

In [39]:
# 4. 
# 드롭아웃, 잔차연결, 정규화 진행
trg = enc_attn_layer_norm(trg + dropout(_trg)) # LayerNorm((256,), eps=1e-05, elementwise_affine=True)
print(trg.shape) #trg = [batch size, trg len, hid dim]

torch.Size([128, 21, 256])


In [40]:
positionwise_feedforward

PositionwiseFeedforwardLayer(
  (fc_1): Linear(in_features=256, out_features=512, bias=True)
  (fc_2): Linear(in_features=512, out_features=256, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [41]:
# 5. 
# positionwise feedforward
_trg = positionwise_feedforward(trg)
print(_trg.shape)

torch.Size([128, 21, 256])


In [42]:
ff_layer_norm

LayerNorm((256,), eps=1e-05, elementwise_affine=True)

In [43]:
# 6. 
# 드롭아웃, 잔차연결, 정규화 진행
trg = ff_layer_norm(trg + dropout(_trg))

# return 값
print(trg.shape) #trg = [batch size, trg len, hid dim]
print(attention.shape) #attention = [batch size, n heads, trg len, src len]

torch.Size([128, 21, 256])
torch.Size([128, 8, 21, 33])


## 2.8 트랜스포머 아키텍처 정의
* 최종적인 전체 트랜스포머(Transformer) 모델을 정의합니다.
* 입력이 들어왔을 때 앞서 정의한 인코더와 디코더를 거쳐 출력 문장을 생성합니다.

In [44]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    # 소스 문장의 <pad> 토큰에 대하여 마스크(mask) 값을 0으로 설정    
    def make_src_mask(self, src):
        
        #src = [batch size, src len]
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    # 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크를 사용
    def make_trg_mask(self, trg):
        
        #trg = [batch size, trg len]
        
        """ (마스크 예시)
        1 0 0 0 0
        1 1 0 0 0
        1 1 1 0 0
        1 1 1 0 0
        1 1 1 0 0
        """
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        """ (마스크 예시)
        1 0 0 0 0
        1 1 0 0 0
        1 1 1 0 0
        1 1 1 1 0
        1 1 1 1 1
        """
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        
        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention

def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx, device):

In [45]:
# 정의
encoder = encoder
decoder = Decoder
decoder = decoder
src_pad_idx = src_pad_idx
trg_pad_idx = trg_pad_idx
device = device

# 소스 문장의 <pad> 토큰에 대하여 마스크(mask) 값을 0으로 설정  
def make_src_mask(self, src):
    src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
    return src_mask

# 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크를 사용        
def make_trg_mask(self, trg):
    trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
    trg_len = trg.shape[1]
    trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
    trg_mask = trg_pad_mask & trg_sub_mask
    return trg_mask

In [46]:
# 이미 앞에서 시행했기 때문에 잘되었는지 shape만 확인
# src_mask = make_src_mask(src)
print(src_mask.shape) #src_mask = [batch size, 1, 1, src len]

# trg_mask = make_trg_mask(trg)
print(trg_mask.shape) #trg_mask = [batch size, 1, trg len, trg len]

# enc_src = encoder(src, src_mask)
print(enc_src.shape) #enc_src = [batch size, src len, hid dim]

print(output.shape) #output = [batch size, trg len, output dim]
print(attention.shape) #attention = [batch size, n heads, trg len, src len]

torch.Size([128, 1, 1, 33])
torch.Size([128, 1, 21, 21])
torch.Size([128, 33, 256])
torch.Size([128, 21, 5893])
torch.Size([128, 8, 21, 33])


# 3. 훈련

In [47]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, HID_DIM, ENC_LAYERS, ENC_HEADS, ENC_PF_DIM, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, HID_DIM, DEC_LAYERS, DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, device)

In [48]:
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

# Transformer 객체 선언
model = Transformer(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [49]:
# 모델 가중치 파라미터 초기화
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

Transformer(
  (encoder): Encoder(
    (tok_embedding): Embedding(7853, 256)
    (pos_embedding): Embedding(100, 256)
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_attention): MultiHeadAttentionLayer(
          (fc_q): Linear(in_features=256, out_features=256, bias=True)
          (fc_k): Linear(in_features=256, out_features=256, bias=True)
          (fc_v): Linear(in_features=256, out_features=256, bias=True)
          (fc_o): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (positionwise_feedforward): PositionwiseFeedforwardLayer(
          (fc_1): Linear(in_features=256, out_features=512, bias=True)
          (fc_2): Linear(in_features=512, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
 

## 함수 및 평가 함수 정의

In [50]:
# Adam optimizer로 학습 최적화
LEARNING_RATE = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0005
    weight_decay: 0
)

* nn.CrossEntropyLoss
    * 다중분류를 위한 대표적인 손실함수
    * nn.LogSoftmax와 nn.NLLLoss의 연산의 조합
        *  nn.LogSoftmax: 신경망 말단의 결과 값들을 확률개념으로 해석하기 위한 Softmax 함수의 결과에 log 값을 취한 연산
        * nn.NLLLoss: nn.LogSoftmax의 log 결과값에 대한 교차 엔트로피 손실 연산(Cross Entropy Loss|Error)

In [51]:
# 손실함수(loss function) 정의
# 뒷 부분의 패딩(padding)에 대해서는 값 무시(<pad> 토큰에 대해서 loss 계산하지 않도록 조건 부여)
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)
criterion

CrossEntropyLoss()

In [52]:
# 모델 학습(train) 함수
def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습 모드
    epoch_loss = 0

    # 전체 학습 데이터를 확인하며
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        # 출력 단어의 마지막 인덱스(<eos>)는 제외
        # 입력을 할 때는 <sos>부터 시작하도록 처리
        output, _ = model(src, trg[:,:-1])

        # output: [배치 크기, trg_len - 1, output_dim]
        # trg: [배치 크기, trg_len]

        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        # 출력 단어의 인덱스 0(<sos>)은 제외
        trg = trg[:,1:].contiguous().view(-1)

        # output: [배치 크기 * trg_len - 1, output_dim]
        # trg: [배치 크기 * trg len - 1]

        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산

        # 기울기(gradient) clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # 파라미터 업데이트
        optimizer.step()

        # 전체 손실 값 계산
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [53]:
# 모델 평가(evaluate) 함수
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0

    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            # 출력 단어의 마지막 인덱스(<eos>)는 제외
            # 입력을 할 때는 <sos>부터 시작하도록 처리
            output, _ = model(src, trg[:,:-1])

            # output: [배치 크기, trg_len - 1, output_dim]
            # trg: [배치 크기, trg_len]

            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim)
            # 출력 단어의 인덱스 0(<sos>)은 제외
            trg = trg[:,1:].contiguous().view(-1)

            # output: [배치 크기 * trg_len - 1, output_dim]
            # trg: [배치 크기 * trg len - 1]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실 값 계산
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

## 학습(training) 및 검증(validation) 진행
학습 횟수(epoch): 10

In [54]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [55]:
N_EPOCHS = 1
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'transformer_german_to_english.pt')

    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')

	Train Loss: 4.236 | Train PPL: 69.117
	Validation Loss: 3.033 | Validation PPL: 20.764


## 최종 결과 확인

In [56]:
test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.048 | Test PPL:  21.081 |
